# 加载数据集

In [15]:
import torch
import numpy as np
# Dataset是一个抽象类，必须被继承使用
from torch.utils.data import Dataset
# DataLoader用来自动加载数据的
from torch.utils.data import DataLoader

In [16]:
# 准备数据
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        # shape=(多少行，多少列)，由shape[0]求有多少行
        self.len = xy.shape[0] 
    # x_data要除最后一列的几列，y_data要最后一列        
        self.x_data = torch.from_numpy(xy[:,:-1])
        self.y_data = torch.from_numpy(xy[:,[-1]])
    # 使支持下标操作 ，可以把某个数据拿出来       
    def __getitem__(self,index):
        # 返回的是元组        
        return self.x_data[index],self.y_data[index]
    # 使支持求数据数量长度    
    def __len__(self):
        return self.len
dataset = DiabetesDataset('diabetes.csv.gz')
# dataset=dataset传递数据集
# num_workers = 2使用双进程
# bactch_size表示一次前馈返馈更新训练的样本数量
# shuffle = True表示打乱顺序
train_loader = DataLoader(dataset=dataset,batch_size = 32,shuffle = True,num_workers = 2)

In [17]:
# 设计模型
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        # 原始输入维度是8维，最终输出维度是1维，我们要找一个八维空间到一维空间的非线性变换
        # 通过sigmoid函数实现给线性变换添加非线性因子        
        self.linear1 = torch.nn.Linear(8,6)
        self.linear2 = torch.nn.Linear(6,4)
        self.linear3 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self,x):
        x= self.sigmoid(self.linear1(x))
        x= self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x
model = Model()

In [18]:
# 构造损失和优化器
criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)# lr是学习率

D:\software\Anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [19]:
# 训练
# 封装起来还是有多线程的运行时错误怎么办?
if __name__ == '__main__':
     for epoch in range(100):
            # train_loader 是一个可迭代的loader,是先shuffle打乱顺序，然后分组为mini_batch，每一次迭代给出的是一个batch
            # 对于一个可迭代的（iterable）/可遍历的对象（如列表、字符串），enumerate将其组成一个索引序列，利用它可以同时获得索引和值         
            for i, data in enumerate(train_loader, 0):
                # 就是x和y，他们都是张量                
                inputs, labels = data
                # 前馈            
                y_pred = model(inputs)
                loss = criterion(y_pred, labels)
                print(epoch, i, loss.item())

                optimizer.zero_grad()
                # 反馈                
                loss.backward()
                # 更新权重 
                optimizer.step()


RuntimeError: DataLoader worker (pid(s) 17060) exited unexpectedly